# Imports

In [1]:
from src.lib import state
from src.lib import ops

### function to create U

In [2]:
def f_creator(user_input: str): 
    user_input_lower = user_input.lower()
    
    def check_input(in_bit): 
        assert type(in_bit) == list, 'function need to be called with list representing bits, e.g. [1] for 1'
        assert len(in_bit) == 1, f'function operates on one bit, but received {len(in_bit)}'
        assert in_bit[0] == 0 or in_bit[0] == 1, f'just 0 or 1, but received {in_bit[0]}'
    
    if user_input_lower == 'set0': 
        def f(input_bit: list) -> int: 
            check_input(input_bit)
            return 0
    elif user_input_lower == 'id': 
        def f(input_bit: list) -> int: 
            check_input(input_bit)
            return input_bit[0]
    elif user_input_lower == 'not': 
        def f(input_bit: list) -> int: 
            check_input(input_bit)
            return 1 - input_bit[0]           # 1 - 0 = 1 and 1 - 1 = 0
    elif user_input_lower == 'set1': 
        def f(input_bit: list) -> int: 
            check_input(input_bit)
            return 1
    else: 
        raise AssertionError('Wrong input')
    return f

In [3]:
def make_f_unitary(f): 
    return ops.OracleUf(2, f)

In [4]:
def run_deutsch(unitary_function: ops.Operator, display_operation: bool = True): 
    # init | 0 1 > 
    xy = state.bitstring(0, 1)
    if display_operation: 
        print('After init: ', xy)
        
    # first hadamard 
    op = ops.Hadamard(1) * ops.Hadamard(1) 
    xy = op(xy)
    if display_operation: 
        print('After Hadamard 1: ', xy)
    
    # unitary function
    xy = unitary_function(xy)
    if display_operation: 
        print('After unitary_function: ', xy)
    
    # hadamard 
    op = ops.Hadamard(1) * ops.Identity(1)
    xy = op(xy)
    if display_operation: 
        print('After Hadamard 2: ', xy)
    
    try:        # if measuring P = 0 -> AssertionError
        ops.Measure(xy, idx=0, tostate=0) 
        return 0
    except AssertionError: 
        ops.Measure(xy, idx=0, tostate=1)
        return 1

In [5]:
user_in = input('Type function (set0, id, not, set1):')
pure_function = f_creator(user_in)
u_f = make_f_unitary(pure_function)
state_first_bit = run_deutsch(u_f)
if state_first_bit == 0: 
    print(user_in, 'is constant')
else: 
    print(user_in, 'is balanced')

After init:  2-qubit state. Tensor:
[0.+0.j 1.+0.j 0.+0.j 0.+0.j]
After Hadamard 1:  2-qubit state. Tensor:
[ 0.49999997+0.j -0.49999997+0.j  0.49999997+0.j -0.49999997+0.j]
After unitary_function:  2-qubit state. Tensor:
[ 0.49999997+0.j -0.49999997+0.j -0.49999997+0.j  0.49999997+0.j]
After Hadamard 2:  2-qubit state. Tensor:
[ 0.       +0.j  0.       +0.j  0.7071067+0.j -0.7071067+0.j]
id is balanced
